# Questionnaire Score Analysis

This notebook analyzes KOOS/WOMAC questionnaire responses to calculate:
- Pain Score
- Symptom Score
- Subjective Score
- Position Capability Scores (SL Stand, Split Stand, DL Stand, Quadruped, Lying)

## Instructions
1. Copy the JSON array from Supabase (Table Editor → Select row → Copy as JSON)
2. Paste it into the `patient_json` variable below
3. Run all cells to see the detailed score breakdown

In [1]:
import json
import pandas as pd
from IPython.display import display, Markdown

# Paste your JSON data from Supabase here
# Example format: [{"idx":0,"id":40,"user_id":1,"username":"user01","f1":2,"f2":3,...}]

patient_json = '''
[{"idx":0,"id":40,"user_id":1,"username":"user01","f1":2,"f2":3,"f3":3,"f4":3,"f5":3,"f6":2,"f7":2,"f8":1,"f9":2,"f10":1,"f11":1,"f12":1,"f13":2,"f14":1,"f15":2,"f16":2,"f17":2,"p1":2,"p2":2,"p3":2,"p4":2,"p5":2,"p6":2,"p7":2,"p8":1,"p9":2,"sp1":4,"sp2":4,"sp3":4,"sp4":2,"sp5":2,"st2":3,"toe_touch_test":"cannot","completed_at":"2026-01-13 09:16:55.654+00","created_at":"2026-01-18 23:05:12.245437+00","s1":2,"s2":2,"s3":2,"s4":2,"s5":2,"st1":1,"q1":2,"q2":2,"q3":2,"q4":2}]'''

# Parse JSON
patient_data = json.loads(patient_json)[0]  # Get first (and only) record

print(f"\n✓ Loaded questionnaire data for user: {patient_data['username']}")
print(f"  User ID: {patient_data['user_id']}")
print(f"  Completed at: {patient_data['completed_at']}")


✓ Loaded questionnaire data for user: user01
  User ID: 1
  Completed at: 2026-01-13 09:16:55.654+00


## 1. Pain Score Calculation

**Formula:** Average of p1-p9  
**Scale:** 1-4 (1 = no pain, 4 = extreme pain)  
**Normalized:** 0-100 (0 = worst, 100 = best)

In [2]:
# Pain Score (p1-p9)
pain_questions = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']
pain_values = [patient_data[q] for q in pain_questions]
pain_avg = sum(pain_values) / len(pain_values)
pain_score = ((4 - pain_avg) / 3) * 100  # Normalized to 0-100

# Display
pain_df = pd.DataFrame({
    'Question': pain_questions,
    'Score': pain_values
})

display(Markdown("### Pain Questions (p1-p9)"))
display(pain_df)
display(Markdown(f"""
**Calculation:**
- Sum: {sum(pain_values)}
- Average: {pain_avg:.3f}
- Formula: `((4 - {pain_avg:.3f}) / 3) × 100`
- **Pain Score: {pain_score:.1f}/100** (higher is better)
"""))

### Pain Questions (p1-p9)

,Question,Score
0,p1,2
1,p2,2
2,p3,2
3,p4,2
4,p5,2
5,p6,2
6,p7,2
7,p8,1
8,p9,2



**Calculation:**
- Sum: 17
- Average: 1.889
- Formula: `((4 - 1.889) / 3) × 100`
- **Pain Score: 70.4/100** (higher is better)


## 2. Symptom Score Calculation

**Formula:** Average of s1-s5  
**Scale:** 1-4 (1 = never, 4 = always)  
**Normalized:** 0-100 (0 = worst, 100 = best)

In [3]:
# Symptom Score (s1-s5)
symptom_questions = ['s1', 's2', 's3', 's4', 's5']
symptom_values = [patient_data[q] for q in symptom_questions]
symptom_avg = sum(symptom_values) / len(symptom_values)
symptom_score = ((4 - symptom_avg) / 3) * 100  # Normalized to 0-100

# Display
symptom_df = pd.DataFrame({
    'Question': symptom_questions,
    'Score': symptom_values
})

display(Markdown("### Symptom Questions (s1-s5)"))
display(symptom_df)
display(Markdown(f"""
**Calculation:**
- Sum: {sum(symptom_values)}
- Average: {symptom_avg:.3f}
- Formula: `((4 - {symptom_avg:.3f}) / 3) × 100`
- **Symptom Score: {symptom_score:.1f}/100** (higher is better)
"""))

### Symptom Questions (s1-s5)

,Question,Score
0,s1,2
1,s2,2
2,s3,2
3,s4,2
4,s5,2



**Calculation:**
- Sum: 10
- Average: 2.000
- Formula: `((4 - 2.000) / 3) × 100`
- **Symptom Score: 66.7/100** (higher is better)


## 3. Subjective Score Calculation

**Formula:** Average of q1-q4  
**Scale:** 1-4 (1 = not at all, 4 = extremely)  
**Normalized:** 0-100 (0 = worst, 100 = best)

In [4]:
# Subjective Score (q1-q4)
subjective_questions = ['q1', 'q2', 'q3', 'q4']
subjective_values = [patient_data[q] for q in subjective_questions]
subjective_avg = sum(subjective_values) / len(subjective_values)
subjective_score = ((4 - subjective_avg) / 3) * 100  # Normalized to 0-100

# Display
subjective_df = pd.DataFrame({
    'Question': subjective_questions,
    'Score': subjective_values
})

display(Markdown("### Quality of Life Questions (q1-q4)"))
display(subjective_df)
display(Markdown(f"""
**Calculation:**
- Sum: {sum(subjective_values)}
- Average: {subjective_avg:.3f}
- Formula: `((4 - {subjective_avg:.3f}) / 3) × 100`
- **Subjective Score: {subjective_score:.1f}/100** (higher is better)
"""))

### Quality of Life Questions (q1-q4)

,Question,Score
0,q1,2
1,q2,2
2,q3,2
3,q4,2



**Calculation:**
- Sum: 8
- Average: 2.000
- Formula: `((4 - 2.000) / 3) × 100`
- **Subjective Score: 66.7/100** (higher is better)


## 4. Position Capability Scores

**Based on ALGORITHM_DOCUMENTATION.md position mapping:**

### Active Positions:
- **SL Stand (Single-leg standing):** f2, f9, f10, f12
- **Split Stand (Lunges, step-ups):** f1, f3, f5, f6, f7, f8, f9, f14, f16
- **DL Stand (Squats):** f1, f3, f4, f5, f6, f7, f11
- **Quadruped (Four-point kneeling):** f1, f15

### Passive Position:
- **Lying (Supine/Side-lying):** f13, f17

**Formula:** Average of mapped function questions → Normalized 0-100

In [5]:
# Position mappings from algorithm
position_mappings = {
    'SL Stand': ['f2', 'f9', 'f10', 'f12'],
    'Split Stand': ['f1', 'f3', 'f5', 'f6', 'f7', 'f8', 'f9', 'f14', 'f16'],
    'DL Stand': ['f1', 'f3', 'f4', 'f5', 'f6', 'f7', 'f11'],
    'Quadruped': ['f1', 'f15'],
    'Lying': ['f13', 'f17']
}

# Calculate position scores
position_scores = {}
position_details = []

for position, questions in position_mappings.items():
    values = [patient_data[q] for q in questions]
    avg = sum(values) / len(values)
    score = ((4 - avg) / 3) * 100  # Normalized to 0-100
    position_scores[position] = score
    
    position_details.append({
        'Position': position,
        'Questions': ', '.join(questions),
        'Values': values,
        'Sum': sum(values),
        'Average': f"{avg:.3f}",
        'Score (0-100)': f"{score:.1f}"
    })

# Display
display(Markdown("### Position Capability Scores"))

for detail in position_details:
    display(Markdown(f"\n**{detail['Position']}**"))
    display(Markdown(f"- Questions: {detail['Questions']}"))
    display(Markdown(f"- Values: {detail['Values']}"))
    display(Markdown(f"- Sum: {detail['Sum']}, Average: {detail['Average']}"))
    display(Markdown(f"- Formula: `((4 - {detail['Average']}) / 3) × 100`"))
    display(Markdown(f"- **Score: {detail['Score (0-100)']}** (higher = better capability)"))

### Position Capability Scores


**SL Stand**

- Questions: f2, f9, f10, f12

- Values: [3, 2, 1, 1]

- Sum: 7, Average: 1.750

...

- Questions: f13, f17

- Values: [2, 2]

- Sum: 4, Average: 2.000

- Formula: `((4 - 2.000) / 3) × 100`

- **Score: 66.7** (higher = better capability)

## 5. Summary Dashboard

In [6]:
# Summary table
summary_data = {
    'Score Type': [
        'Pain Score',
        'Symptom Score',
        'Subjective Score',
        '',
        'SL Stand',
        'Split Stand',
        'DL Stand',
        'Quadruped',
        'Lying'
    ],
    'Score (0-100)': [
        f"{pain_score:.1f}",
        f"{symptom_score:.1f}",
        f"{subjective_score:.1f}",
        '',
        f"{position_scores['SL Stand']:.1f}",
        f"{position_scores['Split Stand']:.1f}",
        f"{position_scores['DL Stand']:.1f}",
        f"{position_scores['Quadruped']:.1f}",
        f"{position_scores['Lying']:.1f}"
    ],
    'Category': [
        'General',
        'General',
        'General',
        '',
        'Active Position',
        'Active Position',
        'Active Position',
        'Active Position',
        'Passive Position'
    ]
}

summary_df = pd.DataFrame(summary_data)

display(Markdown(f"## Patient Summary: {patient_data['username']}"))
display(summary_df)

display(Markdown("""
### Interpretation:
- **Higher scores (closer to 100)** = Better function, less pain, fewer symptoms
- **Lower scores (closer to 0)** = Worse function, more pain, more symptoms

### Clinical Notes:
- Positions with **lowest scores** indicate areas of difficulty → Target for exercise prescription
- Positions with **highest scores** indicate better capability → Can use more challenging exercises
"""))

## Patient Summary: user01

,Score Type,Score (0-100),Category
0,Pain Score,70.4,General
1,Symptom Score,66.7,General
2,Subjective Score,66.7,General
3,,,
4,SL Stand,75.0,Active Position
5,Split Stand,66.7,Active Position
6,DL Stand,57.1,Active Position
7,Quadruped,66.7,Active Position
8,Lying,66.7,Passive Position



### Interpretation:
- **Higher scores (closer to 100)** = Better function, less pain, fewer symptoms
- **Lower scores (closer to 0)** = Worse function, more pain, more symptoms

### Clinical Notes:
- Positions with **lowest scores** indicate areas of difficulty → Target for exercise prescription
- Positions with **highest scores** indicate better capability → Can use more challenging exercises


## 6. Additional Patient Data

In [7]:
# Other relevant data
display(Markdown("### Other Assessment Data"))
display(Markdown(f"""
- **Toe Touch Test:** {patient_data['toe_touch_test']}
- **Stiffness (st1, st2):** {patient_data['st1']}, {patient_data['st2']}
- **Sport/Recreation (sp1-sp5):** {[patient_data[f'sp{i}'] for i in range(1, 6)]}
"""))

### Other Assessment Data


- **Toe Touch Test:** cannot
- **Stiffness (st1, st2):** 1, 3
- **Sport/Recreation (sp1-sp5):** [4, 4, 4, 2, 2]


---

## Notes for LLM Transition:

These scores will help inform:
1. **Patient capability profiling** - Which positions are safe/challenging?
2. **Exercise targeting** - Which biomechanical issues to prioritize?
3. **Progression planning** - How quickly can patient advance?
4. **Clinical reasoning** - LLM can explain recommendations based on score patterns

**For LLM prompts, consider including:**
- Raw scores (pain, symptom, subjective)
- Position capability rankings
- Specific limitations (toe touch, high-impact restrictions)
- Patient context (sport preferences, quality of life concerns)